In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from IPython.core.display import display
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier, RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, RandomizedSearchCV, \
    cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import scale, StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from surfboard.sound import Waveform
from surfboard.feature_extraction import extract_features_from_waveform
import xgboost as xgb

In [51]:
def extract_features(waveforms, components_list, statistics_list=None):
    """This is an important function. Given a list of Waveform objects, a list of
    Waveform methods in the form of strings and a list of Barrel methods in the
    form of strings, compute the time-independent features resulting. This function
    does multiprocessing.

    Args:
        waveforms (list of Waveform): This is a list of waveform objects
        components_list (list of str/dict): This is a list of the methods which
            should be applied to all the waveform objects in waveforms. If a dict,
            this also contains arguments to the sound.Waveform methods.
        statistics_list (list of str): This is a list of the methods which
            should be applied to all the time-dependent features computed
            from the waveforms.

    Returns:
        pandas DataFrame: pandas dataframe where every row corresponds
            to features extracted for one of the waveforms and columns
            represent individual features.
    """
    output_feats = []
    # waveforms = tqdm(waveforms, desc='Extracting features...')

    for wave in waveforms:
        output_feats.append(
            extract_features_from_waveform(
                components_list, statistics_list, wave
            )
        )

    return pd.DataFrame(output_feats)

Используемые библиотеки:
1) liborsa
2) surfboard
3)

Порядок действий:

    1) Усреднить громкость

    2) librosa.effects.preemphasis (In order to equalize the effect of the propagation)

    3) features:
        2.1)MFCC
        2.2)Energy
        2.3)Pitch
        2.4)formants (1 - 3)
        2.5)spectrum centroid????
        2.6)Zero crossing rate
        2.7)jitter
        2.8)shimmers
        2.7) Mean, max, std (статистика по всем фичам на запись)

In [49]:
# functions to extract features from audio to dataframe
components_list_for_stat_features = [
    'f0_contour', 'log_energy', 'spectral_centroid', 'formants', 'loudness',
    'jitters', 'shimmers', 'zerocrossing',
]
components_list_for_plain_features = [
        'f0_contour', 'log_energy', 'spectral_centroid', 'loudness', 'intensity', 'rms', 'ppe']
mfcc_with_arg = {'mfcc': {'n_mfcc': 16, 'n_fft_seconds': 512 / 48000}}
statistics_list = ['mean', 'std', 'min', 'max', 'skewness', 'kurtosis',
                   'first_derivative_mean', 'first_derivative_std', 'q3_q1_range']
components_list_for_stat_features.append(mfcc_with_arg)
components_list_for_plain_features.append(mfcc_with_arg)

def unpack_features_inplace(feature_dataframe : pd.DataFrame) -> pd.DataFrame:
    result  = pd.DataFrame()
    for column in feature_dataframe.columns:
        if type(feature_dataframe[column][0]) is np.ndarray:
            if column is "mfcc":
                # get 2 - 16 mfcc coef for each window
                features_arr : np.ndarray = feature_dataframe.mfcc[0][2:, 50:250].ravel()
                features_arr = features_arr[np.newaxis]
                length = features_arr.shape[1]
            else:
                features_arr = feature_dataframe[column][0][:, 50:250]
                length = len(features_arr[0])

            print(features_arr.shape)
            df = pd.DataFrame(features_arr, columns=[column + str(i) for i in range(length)])
            result = pd.concat([result, df], axis=1)
        else:
            result = pd.concat([result, feature_dataframe[column]], axis=1)
    return result

def get_features(sound_path : str) -> pd.DataFrame:
    audio, sampling_freq = librosa.load(sound_path, sr=None, res_type='scipy')
    # 512 / sampling_freq
    processed_audio = librosa.effects.preemphasis(audio)
    # Extract dataframe...
    audio = Waveform(signal=processed_audio, sample_rate=sampling_freq)
    feature_df = extract_features(
        waveforms=[audio], components_list=components_list_for_stat_features, statistics_list=statistics_list)
    return feature_df


In [ ]:
# making dataframe
global_audio_path = "../data"
res_dataframe = pd.DataFrame()
classes = []
for i in range(1, 6):
    for root, dirs, files in os.walk(os.path.join(global_audio_path, str(i)), topdown=False):
        print(len(files))
        for name in files:
            important_features = get_features(os.path.join(root, name))
            # display(important_features)
            res_dataframe = res_dataframe.append(important_features, ignore_index=True)
            classes.append(i)

In [23]:
def get_mean_conf_matrix(model, X, y):
    conf_matrix_list_of_arrays = []
    kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
    for train_index, test_index in kf.split(X, y):

       X_train, X_test = X[train_index], X[test_index]
       y_train, y_test = y[train_index], y[test_index]

       model.fit(X_train, y_train)
       conf_matrix = confusion_matrix(y_test, model.predict(X_test))
       conf_matrix_list_of_arrays.append(conf_matrix)
    return np.mean(conf_matrix_list_of_arrays, axis=0)

In [9]:
display(res_dataframe)
res_dataframe.describe()

,f0_contour_mean,f0_contour_std,f0_contour_min,f0_contour_max,f0_contour_skewness,f0_contour_kurtosis,f0_contour_first_derivative_mean,f0_contour_first_derivative_std,f0_contour_q3_q1_range,log_energy,...,mfcc_q3_q1_range_7,mfcc_q3_q1_range_8,mfcc_q3_q1_range_9,mfcc_q3_q1_range_10,mfcc_q3_q1_range_11,mfcc_q3_q1_range_12,mfcc_q3_q1_range_13,mfcc_q3_q1_range_14,mfcc_q3_q1_range_15,mfcc_q3_q1_range_16
0,23.967547,55.219171,0.0,189.119143,1.895990,1.668783,1.570260e-16,27.951188,0.000000,-50.789417,...,15.766265,6.817384,10.575198,5.019678,18.639672,4.687609,3.798230,6.998048,4.952859,7.033751
1,27.045372,71.229321,0.0,259.841201,2.297994,3.420751,-1.652425e-16,29.417767,0.000000,-52.897926,...,23.343090,6.969563,17.334488,9.290317,6.188273,5.173421,7.520164,4.676119,8.310646,5.102108
2,8.724462,32.011158,0.0,209.613961,4.010852,17.134221,0.000000e+00,19.592358,0.000000,-57.779049,...,18.296813,12.634435,14.232127,10.144253,10.668671,12.063463,9.836006,11.097036,10.172810,9.805441
3,28.523561,74.396412,0.0,260.546090,2.255366,3.177505,0.000000e+00,36.652883,0.000000,-50.261565,...,16.871922,4.114951,18.549006,9.859148,6.460007,3.631738,6.366929,4.032610,11.482723,7.560311
4,10.253963,40.387550,0.0,238.921752,3.916850,14.445161,0.000000e+00,36.281940,0.000000,-46.754228,...,31.643030,9.679077,11.189731,7.478577,15.093403,6.569043,5.774323,6.686566,10.186784,7.822491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,0.000000,0.000000,0.0,0.000000,0.000000,-3.000000,0.000000e+00,0.000000,0.000000,-49.614353,...,38.190409,11.919920,19.566975,20.034756,26.156811,9.096250,10.394791,12.184984,5.814711,5.547981
855,11.937603,40.763433,0.0,176.104431,3.170061,8.235236,0.000000e+00,19.115677,0.000000,-50.706402,...,18.682658,14.766144,11.331717,14.411584,21.332257,6.488466,8.219062,12.793879,12.382707,8.606090
856,80.651854,98.998965,0.0,237.845414,0.422148,-1.802225,-1.583382e-16,40.416811,197.180354,-55.082888,...,14.759832,9.662964,14.929921,8.962712,10.297725,8.585087,6.362794,13.534030,7.544165,9.226057
857,47.841428,76.043443,0.0,200.364790,0.980022,-0.996395,0.000000e+00,25.452955,151.191570,-55.624970,...,16.418184,10.921530,20.866169,14.092030,12.207006,9.323483,9.331396,9.245415,10.395979,9.108313


,f0_contour_mean,f0_contour_std,f0_contour_min,f0_contour_max,f0_contour_skewness,f0_contour_kurtosis,f0_contour_first_derivative_mean,f0_contour_first_derivative_std,f0_contour_q3_q1_range,log_energy,...,mfcc_q3_q1_range_7,mfcc_q3_q1_range_8,mfcc_q3_q1_range_9,mfcc_q3_q1_range_10,mfcc_q3_q1_range_11,mfcc_q3_q1_range_12,mfcc_q3_q1_range_13,mfcc_q3_q1_range_14,mfcc_q3_q1_range_15,mfcc_q3_q1_range_16
count,859.000000,859.000000,859.0,859.000000,859.000000,859.000000,8.590000e+02,859.000000,859.000000,859.000000,...,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000,859.000000
mean,32.679361,61.670470,0.0,204.244583,2.089628,8.862409,5.657557e-04,31.836192,50.766291,-55.532670,...,18.861103,8.791814,15.332696,9.794284,9.935873,7.130749,8.320488,7.671520,8.140776,6.906689
std,20.745549,26.378037,0.0,66.311670,2.529656,42.154758,1.185124e-02,12.422068,75.607405,9.849781,...,7.376569,3.533583,5.343398,4.383627,4.568073,2.765017,3.655359,3.099579,3.114858,2.799041
min,0.000000,0.000000,0.0,0.000000,0.000000,-3.000000,-6.161888e-16,0.000000,0.000000,-79.909811,...,4.237615,0.000000,1.744585,0.375644,1.538204,0.600210,0.468612,0.053435,0.875458,1.651371
25%,17.189673,45.091285,0.0,180.123285,0.985625,-0.941569,-7.459766e-17,24.962153,0.000000,-62.464710,...,13.823632,6.141642,11.199074,6.642437,6.529782,5.162255,5.885468,5.281707,5.893342,4.877548
50%,30.608498,63.876307,0.0,226.122932,1.483075,0.337269,0.000000e+00,33.257774,0.000000,-57.157595,...,18.083952,8.316085,15.021816,9.011215,9.128621,6.919869,7.745886,7.202434,7.630065,6.443548
75%,47.769585,83.996671,0.0,251.080904,2.181621,3.039097,7.884106e-17,40.507660,112.032888,-48.948679,...,23.132870,11.031903,19.114989,12.291657,12.208972,8.664798,10.028368,9.899950,9.925875,8.690651
max,96.018157,109.769293,0.0,280.559255,21.166063,446.002222,2.796821e-01,59.362170,224.091046,-29.154129,...,44.057369,23.916194,31.031125,29.180224,28.699110,22.758804,27.303226,22.335853,20.096462,21.351741


In [10]:
# check classes
res_dataframe.fillna(0, inplace=True)
target = pd.Series(classes, name="target")
target.unique()

array([1, 2, 3, 4, 5], dtype=int64)

In [ ]:
# save data
full_df = pd.concat([res_dataframe, target], 1)
full_df.to_csv("features_and_stat_full_2.csv", index=False)

In [3]:
# load old data
data = pd.read_csv("features_and_stat_full_2.csv")
target = data["target"]
res_dataframe = data.drop(columns="target")
display(res_dataframe)
display(target)

,f0_contour_mean,f0_contour_std,f0_contour_min,f0_contour_max,f0_contour_skewness,f0_contour_kurtosis,f0_contour_first_derivative_mean,f0_contour_first_derivative_std,f0_contour_q3_q1_range,log_energy,...,mfcc_q3_q1_range_7,mfcc_q3_q1_range_8,mfcc_q3_q1_range_9,mfcc_q3_q1_range_10,mfcc_q3_q1_range_11,mfcc_q3_q1_range_12,mfcc_q3_q1_range_13,mfcc_q3_q1_range_14,mfcc_q3_q1_range_15,mfcc_q3_q1_range_16
0,23.967547,55.219171,0.0,189.119143,1.895990,1.668783,1.570260e-16,27.951188,0.000000,-50.789417,...,15.766265,6.817384,10.575198,5.019678,18.639672,4.687609,3.798230,6.998048,4.952859,7.033751
1,27.045372,71.229321,0.0,259.841201,2.297994,3.420751,-1.652425e-16,29.417767,0.000000,-52.897926,...,23.343090,6.969563,17.334488,9.290317,6.188273,5.173421,7.520164,4.676119,8.310646,5.102108
2,8.724462,32.011158,0.0,209.613961,4.010852,17.134221,0.000000e+00,19.592358,0.000000,-57.779049,...,18.296813,12.634435,14.232127,10.144253,10.668671,12.063463,9.836006,11.097036,10.172810,9.805441
3,28.523561,74.396412,0.0,260.546090,2.255366,3.177505,0.000000e+00,36.652883,0.000000,-50.261565,...,16.871922,4.114951,18.549006,9.859148,6.460007,3.631738,6.366929,4.032610,11.482723,7.560311
4,10.253963,40.387550,0.0,238.921752,3.916850,14.445161,0.000000e+00,36.281940,0.000000,-46.754228,...,31.643030,9.679077,11.189731,7.478577,15.093403,6.569043,5.774323,6.686566,10.186784,7.822491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,0.000000,0.000000,0.0,0.000000,0.000000,-3.000000,0.000000e+00,0.000000,0.000000,-49.614353,...,38.190409,11.919920,19.566975,20.034756,26.156811,9.096250,10.394791,12.184984,5.814711,5.547981
855,11.937603,40.763433,0.0,176.104431,3.170061,8.235236,0.000000e+00,19.115677,0.000000,-50.706402,...,18.682658,14.766144,11.331717,14.411584,21.332257,6.488466,8.219062,12.793879,12.382707,8.606090
856,80.651854,98.998965,0.0,237.845414,0.422148,-1.802225,-1.583382e-16,40.416811,197.180354,-55.082888,...,14.759832,9.662964,14.929921,8.962712,10.297725,8.585087,6.362794,13.534030,7.544165,9.226057
857,47.841428,76.043443,0.0,200.364790,0.980022,-0.996395,0.000000e+00,25.452955,151.191570,-55.624970,...,16.418184,10.921530,20.866169,14.092030,12.207006,9.323483,9.331396,9.245415,10.395979,9.108313


0      1
1      1
2      1
3      1
4      1
      ..
854    5
855    5
856    5
857    5
858    5
Name: target, Length: 859, dtype: int64

Feature selection Section

Here we testing methods for dimension reduction

In [11]:
# Best results overall
# feature names to drop (based on MI decomposition method)
res = mutual_info_classif(res_dataframe, target, random_state=42, n_neighbors=30)
df_res = pd.Series(res)
sorted_df = df_res.sort_values(ascending=False)
index_to_del : pd.Int64Index = sorted_df[sorted_df < 0.1].index
print(index_to_del.shape)
columns_to_del = [res_dataframe.columns[i] for i in index_to_del]
for c in columns_to_del:
    print(c)
# features_cleared_df = res_dataframe.drop(columns=)
# features_cleared_df

(140,)
f0_contour_q3_q1_range
apq3Shimmer
mfcc_q3_q1_range_13
mfcc_mean_13
localShimmer
spectral_centroid_skewness
mfcc_kurtosis_5
mfcc_q3_q1_range_5
mfcc_first_derivative_std_1
mfcc_std_14
spectral_centroid_mean
mfcc_kurtosis_1
mfcc_max_4
mfcc_kurtosis_7
mfcc_std_8
mfcc_mean_5
spectral_centroid_min
mfcc_max_16
mfcc_mean_15
mfcc_std_10
mfcc_first_derivative_std_11
mfcc_mean_14
mfcc_first_derivative_std_8
mfcc_min_4
mfcc_min_6
localdbShimmer
mfcc_q3_q1_range_10
mfcc_mean_12
mfcc_max_12
mfcc_min_15
spectral_centroid_first_derivative_std
mfcc_skewness_7
f0_contour_max
mfcc_q3_q1_range_2
mfcc_std_12
mfcc_q3_q1_range_16
mfcc_first_derivative_mean_3
mfcc_kurtosis_11
mfcc_kurtosis_13
mfcc_mean_16
mfcc_std_9
mfcc_first_derivative_std_5
mfcc_skewness_6
mfcc_first_derivative_std_10
mfcc_min_10
mfcc_min_2
mfcc_skewness_2
mfcc_q3_q1_range_15
num_zerocrossings
mfcc_kurtosis_4
mfcc_min_8
mfcc_max_2
mfcc_mean_4
mfcc_skewness_1
mfcc_skewness_12
f3
mfcc_first_derivative_mean_14
mfcc_first_derivative_st

In [53]:
# k = 90 for 1st set
# k = 110 for 2nd set
# k = 200
select_st = SelectKBest(mutual_info_classif, k=120)
features_cleared_df = select_st.fit_transform(res_dataframe, target)
pd.DataFrame(features_cleared_df)

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,23.967547,55.219171,189.119143,1.895990,1.668783,27.951188,0.000000,-50.789417,10165.480300,4205.444653,...,15.766265,6.817384,10.575198,5.019678,18.639672,4.687609,3.798230,6.998048,4.952859,7.033751
1,27.045372,71.229321,259.841201,2.297994,3.420751,29.417767,0.000000,-52.897926,8238.646944,4965.301582,...,23.343090,6.969563,17.334488,9.290317,6.188273,5.173421,7.520164,4.676119,8.310646,5.102108
2,8.724462,32.011158,209.613961,4.010852,17.134221,19.592358,0.000000,-57.779049,10573.577648,2372.872379,...,18.296813,12.634435,14.232127,10.144253,10.668671,12.063463,9.836006,11.097036,10.172810,9.805441
3,28.523561,74.396412,260.546090,2.255366,3.177505,36.652883,0.000000,-50.261565,8503.507631,5009.949377,...,16.871922,4.114951,18.549006,9.859148,6.460007,3.631738,6.366929,4.032610,11.482723,7.560311
4,10.253963,40.387550,238.921752,3.916850,14.445161,36.281940,0.000000,-46.754228,10341.238850,4201.536338,...,31.643030,9.679077,11.189731,7.478577,15.093403,6.569043,5.774323,6.686566,10.186784,7.822491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,0.000000,0.000000,0.000000,0.000000,-3.000000,0.000000,0.000000,-49.614353,8274.023605,4547.170923,...,38.190409,11.919920,19.566975,20.034756,26.156811,9.096250,10.394791,12.184984,5.814711,5.547981
855,11.937603,40.763433,176.104431,3.170061,8.235236,19.115677,0.000000,-50.706402,9779.106757,4538.508236,...,18.682658,14.766144,11.331717,14.411584,21.332257,6.488466,8.219062,12.793879,12.382707,8.606090
856,80.651854,98.998965,237.845414,0.422148,-1.802225,40.416811,197.180354,-55.082888,10840.344698,3517.150012,...,14.759832,9.662964,14.929921,8.962712,10.297725,8.585087,6.362794,13.534030,7.544165,9.226057
857,47.841428,76.043443,200.364790,0.980022,-0.996395,25.452955,151.191570,-55.624970,10787.963161,2634.707493,...,16.418184,10.921530,20.866169,14.092030,12.207006,9.323483,9.331396,9.245415,10.395979,9.108313


In [39]:
pca_ = PCA(n_components='mle', whiten=True, random_state=42)
features_cleared_df = pca_.fit_transform(res_dataframe)
print("Components: ")
print(pca_.n_components)
print("Var: ")
print(pca_.explained_variance_ratio_)
print("Dim of data: ")
pd.DataFrame(features_cleared_df)

Components: 
mle
Var: 
[9.31892957e-01 3.06630407e-02 1.53402566e-02 9.76171428e-03
 7.03184323e-03 1.43844174e-03 1.03285113e-03 8.94161627e-04
 6.89213744e-04 5.31252539e-04 4.34096033e-04 1.41510449e-04
 5.02834704e-05 3.14587058e-05 1.69566299e-05 1.13799716e-05
 5.50875443e-06 3.63428612e-06 3.31432591e-06 2.76885923e-06
 2.03455642e-06 1.64424169e-06 1.55359180e-06 1.36903476e-06
 1.27255987e-06 1.13541670e-06 1.09165650e-06 1.02807865e-06
 9.59402057e-07 8.35611023e-07 7.67073798e-07 7.22557666e-07
 6.10982627e-07 5.79762035e-07 5.59130410e-07 4.97779244e-07
 4.64097777e-07 4.29026479e-07 4.19806483e-07 3.83149428e-07
 3.48465844e-07 3.15226987e-07 3.02643730e-07 2.82474098e-07
 2.77690634e-07 2.57219402e-07 2.47982052e-07 2.35972545e-07
 2.24140561e-07 2.03176403e-07 1.93094973e-07 1.82324773e-07
 1.74894116e-07 1.52160724e-07 1.32815505e-07 1.23353155e-07
 1.14323505e-07 1.04079091e-07 9.82803279e-08 8.55237552e-08
 8.33585098e-08 7.44160398e-08 7.06706902e-08 6.03891760e-08
 

,0,1,2,3,4,5,6,7,8,9,...,168,169,170,171,172,173,174,175,176,177
0,-0.654417,-0.718141,0.030132,-1.072910,0.708123,-0.020596,-0.017338,-0.427760,-1.085239,0.421242,...,-0.437299,-0.381024,-0.126177,-0.536485,0.179708,-0.767484,-0.257893,-0.294117,0.220843,-1.061629
1,-0.960351,-0.933735,0.608160,0.640322,-1.252787,-0.458721,-0.934445,-0.248745,0.559189,-0.358836,...,-0.735460,0.436552,0.642516,0.032466,-0.472319,-0.327293,2.469069,-0.011533,0.354069,0.329528
2,2.130659,-0.445539,-1.069364,2.001907,0.002100,-0.344728,0.760424,0.515909,-0.302349,-1.300116,...,-1.014061,-0.712787,-1.197466,-0.544358,-1.696698,-0.350922,0.681480,-1.449786,-0.447226,-0.741498
3,-0.826648,-1.030432,0.703325,1.846412,-0.696029,-0.377974,-0.469339,0.938330,-0.320545,1.094599,...,-0.780647,0.290459,0.547129,0.114253,0.444668,0.542399,2.306907,-0.170407,-0.740525,0.703598
4,0.224253,0.367593,1.689470,1.139996,0.136001,0.675708,-0.211493,-0.738881,0.135077,1.742833,...,-0.149573,0.876098,0.317102,-0.119419,0.075978,-0.108216,-0.775660,-0.725422,-0.774697,-1.059167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,-0.614313,-1.181972,-0.197831,-0.360738,-1.427620,-0.512256,-0.997067,-0.836648,0.637792,2.093875,...,-0.419476,-0.593385,1.493381,-0.106604,0.350686,-0.607659,-0.108891,-0.009390,0.306686,1.029953
855,-0.193450,0.371001,2.198106,-0.610272,-0.579873,0.303826,0.213992,-0.150671,-0.487775,1.583659,...,-0.349163,0.218169,-0.310940,1.234166,-0.170619,-0.965031,-0.471036,-0.031749,-0.476844,1.173416
856,-0.168456,1.562769,-0.109923,0.865072,-0.549880,0.162355,0.367857,-0.156663,-0.076021,-0.139445,...,0.424871,0.301903,0.788551,-0.019423,0.644747,0.001294,2.049701,-1.184725,-1.564025,1.264369
857,2.375942,0.096376,-0.192177,-0.799078,-0.359728,1.401976,0.695170,1.090006,-0.552889,-0.526679,...,0.479583,-2.924415,-0.018133,1.367868,-1.141636,1.469116,1.142611,0.832079,-0.312097,0.053825


Classification section

Here we try to choose the best classifier and params for it

In [ ]:
n_classes = 5
classifier = GaussianMixture(n_components=n_classes, covariance_type='full', max_iter=100)
cv = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
X = scale(features_cleared_df)
y = target
conf_matrix_list_of_arrays = []
for train_index, test_index in cv.split(X, y):
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]
   classifier.means_ = np.array([X_train[y_train == i].mean(axis=0) for i in range(1, n_classes + 1)])
   classifier.fit(X_train)
   y_train_pred = classifier.predict(X_test)
   print(y_train_pred.shape)
   conf_matrix = confusion_matrix(y_test, y_train_pred)
   conf_matrix_list_of_arrays.append(conf_matrix)
np.mean(conf_matrix_list_of_arrays, axis=0)

In [45]:
X = scale(features_cleared_df)
estimator_ = xgb.XGBClassifier()
# y_pred = cross_val_predict(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
# conf_mat = confusion_matrix(target, y_pred)
# print(conf_mat)
get_mean_conf_matrix(estimator_, X, target)

array([[29.2,  0.2,  2.4,  0. ,  2.6],
       [ 0. , 34.6,  0.6, 13.4,  3.2],
       [ 4.4,  3.4, 24. ,  0. ,  2.6],
       [ 0. , 16.2,  0. ,  0.4,  0.4],
       [ 1. , 12.8,  4.8,  0.4, 15.2]])

In [ ]:
X = scale(res_dataframe)
cv = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
params = {"max_depth" : [3,5,7], "n_estimators" : [100, 200, 400]}
print(params)
estimator_ = GradientBoostingClassifier()
gs = GridSearchCV(estimator_, params, cv=cv,scoring="accuracy", verbose=2, n_jobs=-1)
gs.fit(X, target)
gs.cv_results_

In [27]:
X = scale(features_cleared_df)
estimator_ = KNeighborsClassifier(n_neighbors=3)
# y_pred = cross_val_predict(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
# conf_mat = confusion_matrix(target, y_pred)
# print(conf_mat)
get_mean_conf_matrix(estimator_, X, target)

array([[27.6,  2.8,  3.6,  0. ,  0.4],
       [ 1.4, 38.6,  0. , 10.4,  1.4],
       [ 7. ,  6.8, 17. ,  0.8,  2.8],
       [ 0. , 14.4,  0. ,  2. ,  0.6],
       [ 2.2, 12.8,  2.4,  0.4, 16.4]])

In [25]:
# X = scale(res_dataframe)
X = scale(features_cleared_df)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = LinearSVC(C=0.01, dual=False)
y_pred = cross_val_predict(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
conf_mat = confusion_matrix(target, y_pred)
print(conf_mat)

array([[30.8,  0.6,  1.4,  0. ,  1.6],
       [ 0.6, 41.4,  1.8,  3.2,  4.8],
       [ 4.2,  2.4, 25. ,  0. ,  2.8],
       [ 0.2, 14.2,  1. ,  0. ,  1.6],
       [ 2.4, 12.2,  4.6,  0. , 15. ]])

In [ ]:
X = scale(features_cleared_df)
cv = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
params = {"C" : 10 ** np.arange(-5, 5, dtype = float), "penalty" : ["l1", 'l2'],
          'dual' : [True, False], 'loss' : ['hinge', 'squared_hinge'], 'multi_class' : ['ovr', 'crammer_singer']}
print(params)
estimator_ = LinearSVC()
gs = GridSearchCV(estimator_, params, cv=cv,scoring="accuracy", verbose=2, n_jobs=-1)
gs.fit(X, target)
print(gs.cv_results_)
print(gs.best_params_)

In [26]:
X = scale(features_cleared_df)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = LogisticRegression("l2", C=0.001)
y_pred = cross_val_predict(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
conf_mat = confusion_matrix(target, y_pred)
print(conf_mat)

array([[27.2,  1.8,  4. ,  0. ,  1.4],
       [ 0. , 49.6,  1. ,  0. ,  1.2],
       [ 7.8,  5.6, 19. ,  0. ,  2. ],
       [ 0. , 16.8,  0.2,  0. ,  0. ],
       [ 2.6, 21.4,  7. ,  0. ,  3.2]])

In [42]:
X = scale(features_cleared_df)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = DecisionTreeClassifier(splitter="random")
get_mean_conf_matrix(estimator_, X, target)

array([[24.6,  1.2,  5.8,  0. ,  2.8],
       [ 0.6, 27.2,  2.8, 13.6,  7.6],
       [ 6.8,  3.6, 15.8,  0.2,  8. ],
       [ 0. , 15.6,  0.4,  0. ,  1. ],
       [ 2.6,  9.6,  5.6,  1.8, 14.6]])

In [12]:
X = scale(features_cleared_df)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = MLPClassifier(hidden_layer_sizes=100, learning_rate='adaptive', solver='adam')
y_pred = cross_val_predict(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
conf_mat = confusion_matrix(target, y_pred)
print(conf_mat)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    1.6s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.6s finished


[[152   1   9   0  10]
 [  0 169   2  76  12]
 [ 11  10 134   0  17]
 [  0  78   0   5   2]
 [  6  35  17  11 102]]


In [13]:
X = scale(features_cleared_df)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
estimator_ = RandomForestClassifier(n_estimators=500, min_samples_leaf=2, max_features = 'sqrt',
                                  max_depth= 90, bootstrap=False, random_state=51)
y_pred = cross_val_predict(estimator_, X, target, cv = cv_, n_jobs=-1, verbose=2)
conf_mat = confusion_matrix(target, y_pred)
print(conf_mat)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:    3.2s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.3s finished


[[147   3  14   0   8]
 [  0 183   3  65   8]
 [ 18  16 129   0   9]
 [  0  82   1   0   2]
 [  1  65  26   0  79]]


In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
criterion = ['gini', 'entropy']
class_weight = ['balanced', 'balanced_subsample']
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion' : criterion,
               'class_weight' : class_weight,
               }
X = scale(features_cleared_df)
estimator_ = RandomForestClassifier()
gs = RandomizedSearchCV(estimator_, random_grid, n_iter = 500 ,cv=cv,scoring="accuracy", verbose=2, n_jobs=-1)
gs.fit(X, target)
print(gs.cv_results_)
print(gs.best_params_)

In [51]:
gs.best_score_

0.6392084863321094

# FINAL VERSION OF CLASSIFIER

In [57]:
# best rf {'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 90, 'criterion': 'gini', 'class_weight': 'balanced', 'bootstrap': False}
estimators = [
    ('mlp', MLPClassifier(hidden_layer_sizes=200, learning_rate='adaptive', solver='adam')),
    ("rf", RandomForestClassifier(n_estimators=500, min_samples_leaf=2, max_features = 'sqrt',
                                  max_depth= 90, bootstrap=False, random_state=51)),
    ('lr', LogisticRegression("l2", C=0.01)),
    ('svc', LinearSVC(C=0.01, dual=True, loss='hinge', random_state=51)),
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
scaler = StandardScaler()
X = scaler.fit_transform(features_cleared_df)
# get_mean_conf_matrix(estimator_, X, target)
cv_ = StratifiedKFold(n_splits = 5, shuffle=True, random_state=241)
scores : np.ndarray = cross_val_score(clf, X, target, cv = cv_, n_jobs=-1, verbose=2)
# score ~ 0.81
print("Min - " , np.min(scores))
print("Max - " , np.max(scores))
print("Mean - " , scores.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   29.4s remaining:   19.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   29.7s finished


Min -  0.6976744186046512
Max -  0.7719298245614035
Mean -  0.7241534067727459


In [11]:
gs.best_params_

{'C': 1.0, 'degree': 1}

# PROCESSING AND PREDICTING OF TEST SET

In [ ]:
# обработка тестовой выборки
global_audio_path = "../data"
test_set_ = pd.DataFrame()
classes = []
names = []
for root, dirs, files in os.walk(os.path.join(global_audio_path, "test_set"), topdown=False):
    print(len(files))
    for name in files:
        print(name)
        names.append(name)
        # important_features = get_features(os.path.join(root, name))
        # # display(important_features)
        # test_set_ = test_set_.append(important_features, ignore_index=True)

In [56]:
test_set_.fillna(0, inplace=True)
cleared_test = select_st.transform(test_set_)
pd.DataFrame(cleared_test)

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,51.761980,89.626923,263.145004,1.162555,-0.626445,41.573526,96.804841,-62.017670,9803.318066,4962.924887,...,10.901967,4.739079,15.589056,5.605010,4.594278,4.318587,6.782056,3.646079,4.366246,3.755388
1,0.000000,0.000000,0.000000,0.000000,-3.000000,0.000000,0.000000,-31.887675,10485.265788,4351.573151,...,23.448857,2.489312,17.148796,0.878775,14.743590,1.864468,11.149280,14.526752,8.048283,9.064811
2,35.397077,55.927989,185.066697,1.127367,-0.266672,27.167014,95.072718,-67.902363,10060.717709,2202.829904,...,22.212913,15.895349,17.173086,9.584425,11.400564,11.709084,10.173433,9.181219,11.903566,11.237692
3,27.364267,72.333238,253.814503,2.285329,3.279329,42.248805,0.000000,-54.413660,10041.557461,3551.504570,...,27.097754,11.603255,17.406931,12.578803,8.694874,9.359720,7.738621,9.511879,9.678088,9.846202
4,3.855143,31.463190,263.382740,8.039779,62.645896,20.013667,0.000000,-43.799506,8590.485127,5248.936163,...,23.299897,3.042001,17.594632,12.641233,6.904490,1.457584,15.749333,1.141038,13.343830,5.371933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,21.645368,62.314291,244.153474,2.614622,5.101378,31.507584,0.000000,-47.775701,8804.096340,4881.365892,...,19.961931,6.821790,17.819378,8.581701,8.480812,10.313920,13.023061,9.773853,4.356276,6.579113
96,53.113282,68.197958,182.413424,0.619746,-1.395437,25.466492,108.757727,-63.401125,10274.223266,2430.769629,...,20.368541,16.407760,16.499924,15.113135,9.278211,11.257612,11.646778,10.147658,9.637830,11.846105
97,17.493321,38.744720,120.326275,1.791117,1.282873,16.067463,0.000000,-61.205179,10890.427658,3964.634509,...,22.189019,10.891023,9.854234,13.016143,7.832365,6.857023,4.914176,7.695723,9.991849,6.155418
98,49.522587,71.543061,199.103020,0.818116,-1.187261,31.616064,133.485978,-58.436573,10839.581364,4134.331942,...,23.562635,6.593991,9.612580,6.661726,12.378902,8.053707,6.132192,6.025013,9.404975,7.740971


In [ ]:
estimators = [
    ('mlp', MLPClassifier(hidden_layer_sizes=200, learning_rate='adaptive', solver='adam')),
    ("rf", RandomForestClassifier(n_estimators=500, min_samples_leaf=2, max_features = 'sqrt',
                                  max_depth= 90, bootstrap=False, random_state=51)),
    ('lr', LogisticRegression("l2", C=0.01)),
    ('svc', LinearSVC(C=0.01, dual=True, loss='hinge', random_state=51)),
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
scaler = StandardScaler()
X_test = scaler.fit_transform(features_cleared_df)
X_train = scaler.transform(cleared_test)
clf.fit(X_test, target)
print("Done")

In [66]:
pred = clf.predict(X_train)
pred_proba = clf.predict_proba(X_train)
print(pred)
print(pred_proba)
max_prob_for_file = np.round(np.max(pred_proba, 1), 2)
print(max_prob_for_file)
df_res = pd.DataFrame({"file name" : names, "predicted class" : pred, "prob" : max_prob_for_file})

[2 1 2 1 1 5 2 3 2 4 3 5 3 4 1 5 2 1 5 2 2 1 3 1 2 2 1 5 4 5 1 3 3 3 3 5 3
 1 2 1 1 1 4 2 1 1 2 1 1 3 1 5 4 5 5 4 4 2 3 2 1 2 3 2 2 2 1 2 3 2 2 3 2 3
 2 2 4 1 3 1 4 5 2 1 1 2 2 1 3 1 2 5 5 2 5 3 2 5 4 2]
[[4.64429759e-03 6.65166763e-01 1.06731766e-02 2.39461313e-01
  8.00544493e-02]
 [9.56428471e-01 5.99869098e-04 3.31696958e-02 1.17796184e-03
  8.62400205e-03]
 [9.72500557e-03 7.45536266e-01 8.24631415e-03 3.32192163e-02
  2.03273198e-01]
 [9.67989146e-01 9.71658237e-04 1.14113045e-02 5.65064693e-04
  1.90628265e-02]
 [8.19249552e-01 2.95248481e-03 1.12085098e-01 2.77942037e-03
  6.29334452e-02]
 [1.12400092e-02 2.80256300e-01 2.24942448e-01 9.24475861e-02
  3.91113657e-01]
 [2.86392204e-03 6.99163976e-01 1.28741807e-02 2.17722593e-01
  6.73753289e-02]
 [4.09187073e-02 3.29238940e-03 9.11052457e-01 2.87966061e-03
  4.18567861e-02]
 [1.54933880e-02 5.65730364e-01 2.40589548e-02 1.37654468e-01
  2.57062825e-01]
 [4.53830090e-03 1.52795632e-01 1.51588912e-02 8.00924050e-01
  2.65831262e-

In [68]:
display(df_res)
df_res.to_csv("results.csv", index=False)

,file name,predicted class,prob
0,track_1.wav,2,0.67
1,track_10.wav,1,0.96
2,track_100.wav,2,0.75
3,track_11.wav,1,0.97
4,track_12.wav,1,0.82
...,...,...,...
95,track_95.wav,3,0.92
96,track_96.wav,2,0.79
97,track_97.wav,5,0.78
98,track_98.wav,4,0.68
